<a href="https://colab.research.google.com/github/sunamkundal01/machine-learning-tutorials/blob/main/fever_prediction_model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Collect data from chatgpt and store it in excel
##Text: "I have been feeling hot and shivering at night."
##Label: 1 (fever)
##Text: "I have a sore throat but no temperature."
##Label: 0 (no fever)

In [23]:
from google.colab import files
uploaded = files.upload()



In [24]:
import pandas as pd
df = pd.read_csv('fever_detection_dataset.xlsx')


UnicodeDecodeError: 'utf-8' codec can't decode byte 0x8f in position 22: invalid start byte

In [6]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, classification_report
import string
import nltk
from nltk.corpus import stopwords


In [7]:
nltk.download('stopwords')
stop_words = set(stopwords.words('english'))


[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [8]:
df = pd.read_excel('fever_detection_dataset.xlsx')

In [9]:
df.head()

,Symptom Description,Fever Indication
0,I have chills and feel light-headed frequently.,1
1,"I have a mild headache, but my energy levels a...",0
2,"My body feels weak, and I have been sweating p...",1
3,"I feel a bit light-headed, but my temperature ...",0
4,"My body feels weak, and I have been sweating p...",1


# Data Preprocessing

In [10]:

def preprocess_text(text):
    # Lowercase the text
    text = text.lower()
    # Remove punctuation
    text = text.translate(str.maketrans('', '', string.punctuation))
    # Remove stopwords
    text = " ".join([word for word in text.split() if word not in stop_words])
    return text

In [11]:
df['Processed Text'] = df['Symptom Description'].apply(preprocess_text)

In [12]:
df.head()


,Symptom Description,Fever Indication,Processed Text
0,I have chills and feel light-headed frequently.,1,chills feel lightheaded frequently
1,"I have a mild headache, but my energy levels a...",0,mild headache energy levels normal
2,"My body feels weak, and I have been sweating p...",1,body feels weak sweating profusely
3,"I feel a bit light-headed, but my temperature ...",0,feel bit lightheaded temperature feels fine
4,"My body feels weak, and I have been sweating p...",1,body feels weak sweating profusely


In [13]:
df.size

1500

#Vectorization with TF-IDF

In [14]:
vectorizer = TfidfVectorizer(max_df=0.8, ngram_range=(1, 2))
X = vectorizer.fit_transform(df['Processed Text'])
y = df['Fever Indication']


# Split data

In [15]:

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)


#Model Training with Naive Bayes

In [16]:
# Initialize the Naive Bayes model
from sklearn.naive_bayes import MultinomialNB
nb_model = MultinomialNB()

# Train the model
nb_model.fit(X_train, y_train)


MultinomialNB()

In [17]:
# Make predictions on the test set
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix # Import the confusion_matrix function
y_pred = nb_model.predict(X_test)

# Calculate accuracy
accuracy = accuracy_score(y_test, y_pred)
print("Accuracy:", accuracy)

# Print classification report
print("Classification Report:\n", classification_report(y_test, y_pred))

# Confusion Matrix
conf_matrix = confusion_matrix(y_test, y_pred)
print("Confusion Matrix:\n", conf_matrix)


Accuracy: 1.0
Classification Report:
               precision    recall  f1-score   support

           0       1.00      1.00      1.00        47
           1       1.00      1.00      1.00        53

    accuracy                           1.00       100
   macro avg       1.00      1.00      1.00       100
weighted avg       1.00      1.00      1.00       100

Confusion Matrix:
 [[47  0]
 [ 0 53]]


# Now  predict whether a given review indicates fever or not

In [18]:

def predict_fever(review):
    # Preprocess the review text
    processed_review = preprocess_text(review)

    # Vectorize the review
    review_vectorized = vectorizer.transform([processed_review])

    # Predict using the trained Naive Bayes model
    prediction = nb_model.predict(review_vectorized)

    # Display the result
    if prediction == 1:
        return "Fever"
    else:
        return "No Fever"

# Example of how to get the prediction
new_review = input("Enter the patient's symptom description: ")
prediction_result = predict_fever(new_review)
print(f"Prediction: {prediction_result}")










Enter the patient's symptom description: i have a strong headache
Prediction: No Fever


In [19]:
prediction_result


'No Fever'

In [20]:
res=predict_fever('I have a sore throat but no other symptoms.')

In [21]:
res

'No Fever'

hence our model perdicts the fever based on the patient review .


In [25]:
df

,Symptom Description,Fever Indication,Processed Text
0,I have chills and feel light-headed frequently.,1,chills feel lightheaded frequently
1,"I have a mild headache, but my energy levels a...",0,mild headache energy levels normal
2,"My body feels weak, and I have been sweating p...",1,body feels weak sweating profusely
3,"I feel a bit light-headed, but my temperature ...",0,feel bit lightheaded temperature feels fine
4,"My body feels weak, and I have been sweating p...",1,body feels weak sweating profusely
...,...,...,...
495,I feel a little congested with a runny nose.,0,feel little congested runny nose
496,I've had constant body aches and feel unusuall...,1,ive constant body aches feel unusually tired
497,"My throat feels scratchy, and I have a mild he...",0,throat feels scratchy mild headache
498,I have chills and feel light-headed frequently.,1,chills feel lightheaded frequently


# Added some Question to  it  for the preddiction

In [26]:
def ask_questions():
    # List of questions
    questions = [
        "Are you experiencing chills or shivering? (Yes/No): ",
        "Do you have a sore throat? (Yes/No): ",
        "Do you feel extremely tired? (Yes/No): ",
        "Are you experiencing body aches? (Yes/No): ",
        "Do you feel warm or have a high temperature? (Yes/No): "
    ]

    responses = []

    # Ask each question
    for question in questions:
        answer = input(question).strip().lower()
        if answer == "yes":
            responses.append("Yes")
        else:
            responses.append("No")

    return responses

In [27]:
def predict_fever_from_questions(responses):
    # Map responses to text-based symptom descriptions
    symptom_descriptions = {
        "Yes": "I am experiencing this symptom.",
        "No": "I am not experiencing this symptom."
    }

    predictions = []

    for response in responses:
        # Convert response to a symptom description
        review = symptom_descriptions[response]

        # Preprocess and vectorize the response
        processed_review = preprocess_text(review)
        review_vectorized = vectorizer.transform([processed_review])

        # Make a prediction using the trained model
        prediction = nb_model.predict(review_vectorized)
        predictions.append(prediction[0])  # Append 1 (Fever) or 0 (No Fever)

    # Determine the final result based on majority
    fever_count = sum(predictions)
    if fever_count > len(predictions) / 2:
        return "Fever"
    else:
        return "No Fever"


In [28]:
# Interactive session
responses = ask_questions()
final_prediction = predict_fever_from_questions(responses)
print(f"Final Prediction: {final_prediction}")

Are you experiencing chills or shivering? (Yes/No): yes
Do you have a sore throat? (Yes/No): yes
Do you feel extremely tired? (Yes/No): no
Are you experiencing body aches? (Yes/No): no
Do you feel warm or have a high temperature? (Yes/No): yes
Final Prediction: Fever
